# Dataset preparation

Ci sono degli id in parent_comment_id che non appaiono in id_to_auth:
+ potrei aggiungere un filtro anche per quelli e interrompere le catene.

In [190]:
# Import libreries.
import pandas as pd
import numpy as np

In [191]:
# Import dataframes.
df_posts = pd.read_csv("c:/Users/HP/Desktop/UNI/LM_1/SNA/SNA/data/total_post_2.csv")
df_comments = pd.read_csv("c:/Users/HP/Desktop/UNI/LM_1/SNA/SNA/data/total_comm_2.csv")

C:\Users\HP\AppData\Local\Temp\ipykernel_2432\796903101.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_posts = pd.read_csv("c:/Users/HP/Desktop/UNI/LM_1/SNA/SNA/data/total_post_2.csv")


In [192]:
# Every id must be a string.
df_comments['id'] = df_comments['id'].astype(str)
df_posts['id'] = df_posts['id'].astype(str)

In [193]:
# Remove errors in id.
def check_id(x):
    if x.startswith('t3_') or x.startswith('t1_'):
        return x[:3]
    else: return x
df_comments['id'] = df_comments['id'].apply(check_id)
df_posts['id'] = df_posts['id'].apply(check_id)

In [194]:
# Drop all Nan values.
df_comments = df_comments.dropna(subset=['link_id']).copy()

In [195]:
# Convert created_utc columns to datetime
df_comments['created_utc'] = pd.to_datetime(df_comments['created_utc'], unit='s')
df_posts['created_utc'] = pd.to_datetime(df_posts['created_utc'], unit='s' , errors='coerce')

combined_df = pd.concat([df_comments[['author', 'created_utc']], df_posts[['author', 'created_utc']]])

# Group by author and calculate the first and last activity date
user_activity = combined_df.groupby('author')['created_utc'].agg(['min', 'max'])

# Calculate the duration of activity in days
user_activity['duration'] = (user_activity['max'] - user_activity['min']).dt.days

# Filter users active for at least 14 days
active_users = list(user_activity[user_activity['duration'] >= 14].index)

C:\Users\HP\AppData\Local\Temp\ipykernel_2432\495296260.py:3: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_posts['created_utc'] = pd.to_datetime(df_posts['created_utc'], unit='s' , errors='coerce')


In [196]:
flair_df = pd.concat([df_comments[['author', 'author_flair_css_class']], df_posts[['author', 'author_flair_css_class']]]).drop_duplicates()

# Keeps users with mtbi flag.
mbti = ['intj', 'intp', 'entj', 'entp', 'infj', 'infp', 'enfj', 'enfp', 'istj', 'isfj', 'estj', 'esfj', 'istp', 
        'isfp', 'estp', 'esfp']
flair_df = flair_df[flair_df['author_flair_css_class'].isin(mbti)]

# Create a list with all duplicated authors.
duplicates = flair_df.duplicated(subset=['author'], keep=False)

# Drop rows where duplicates are found.
flair_df = flair_df[~duplicates]

# List of users with correct flair class.
correct_flair_users = list(flair_df['author'])

### Filters
+ Inactivity filter (through active_users)
+ Flair filter (for every author that has more than 1 flair class/ that has no flair class)
+ Deleted filter (for every deleted author)

In [197]:
# Inactivity filter.
df_comments['inactivity_filter'] = df_comments['author'].isin(active_users)
df_posts['inactivity_filter'] = df_posts['author'].isin(active_users)

In [198]:
# Flair filter.
df_comments['flair_filter'] = df_comments['author'].isin(correct_flair_users)
df_posts['flair_filter'] = df_posts['author'].isin(correct_flair_users)

In [199]:
# Deleted filter.
df_comments['deleted_filter'] = df_comments['author'] != '[deleted]'
df_posts['deleted_filter'] = df_posts['author'] != '[deleted]'

In [200]:
df_comments['filter'] = df_comments['inactivity_filter'] & df_comments['flair_filter'] & df_comments['deleted_filter']
df_posts['filter'] = df_posts['inactivity_filter'] & df_posts['flair_filter'] & df_posts['deleted_filter']

In [201]:
df_comments = df_comments.drop(columns=['Unnamed: 0'])
df_posts = df_posts.drop(columns=['Unnamed: 0'])

Every row with a False in filter column must be neglected. We now need to link the comment chains back in order to mantain the interactions between authors even if the comment chain is interrupted by an invalid id.

In [202]:
df_posts

,author,author_flair_css_class,created_utc,id,inactivity_filter,flair_filter,deleted_filter,filter
0,rntksi,infp,2010-12-30 23:00:16,etvvt,True,False,True,False
1,rntksi,infp,2010-12-30 23:02:24,etvwm,True,False,True,False
2,0vertime,NaN,2011-03-10 19:01:20,g1d7b,True,False,True,False
3,rntksi,NaN,2011-03-11 06:41:04,g1rf3,True,False,True,False
4,0vertime,NaN,2011-03-14 18:10:08,g3tg3,True,False,True,False
...,...,...,...,...,...,...,...,...
192589,[deleted],NaN,2022-12-31 23:29:28,1006j8c,True,False,False,False
192590,High__IQ__ENTJ,entj,2022-12-31 23:30:06,1006jo0,False,True,True,False
192591,chaos-seeker320,estj,2022-12-31 23:31:53,1006kz8,False,True,True,False
192592,Ash__Williams,NaN,2022-12-31 23:32:52,1006lmd,True,True,True,True


In [203]:
df_comments

,author_flair_css_class,created_utc,id,author,parent_id,link_id,inactivity_filter,flair_filter,deleted_filter,filter
0,NaN,2011-01-11 21:41:34,c1ccls5,Lamzn6,t3_etvwm,t3_etvwm,True,True,True,True
1,NaN,2011-01-12 09:38:53,c1cf98n,rntksi,t1_c1ccls5,t3_etvwm,True,False,True,False
2,NaN,2011-02-04 08:07:20,c1ffx3l,crystalita,t3_etvwm,t3_etvwm,True,True,True,True
3,NaN,2011-02-04 08:19:33,c1ffy0u,rntksi,t1_c1ffx3l,t3_etvwm,True,False,True,False
4,NaN,2011-03-11 06:37:08,c1ka7we,rntksi,t3_g1d7b,t3_g1d7b,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...
2961255,intp,2022-12-31 23:54:04,j2fwv5e,Deep_Craft_3760,t3_1003lfy,t3_1003lfy,True,True,True,True
2961256,istp,2022-12-31 23:54:28,j2fwx7a,Zealousideal_Mix_629,t1_j2fsrf2,t3_zzu1je,True,True,True,True
2961257,estj,2022-12-31 23:56:52,j2fx8tx,chaos-seeker320,t1_j2fvunu,t3_1006e8z,False,True,True,False
2961258,NaN,2022-12-31 23:58:42,j2fxhsa,[deleted],t1_j2fwx7a,t3_zzu1je,True,False,False,False


In [204]:
filtered_out_comm = df_comments[~df_comments['filter']]['id'].tolist()
filtered_out_post = df_posts[~df_posts['filter']]['id'].tolist()
filtered_out = list(set(filtered_out_comm) | set(filtered_out_post))

If the parent_id of a certain comment is inside filtered_out_ids we need to link it to the parent comment of the invalid one.

In [205]:
# df_comments + df_posts.
df_full = pd.concat([df_comments[['id','author','author_flair_css_class']], df_posts[['id','author','author_flair_css_class']]], axis=0)

In [206]:
# Dictionary that links id to author.
id_to_auth = df_full.set_index('id')['author'].to_dict()

In [207]:
df_comments['parent_comment_id'] = df_comments['parent_id'].apply(lambda x: x[3:])

In [208]:
ids = df_comments['parent_comment_id'].unique()
ids_full = df_full['id'].unique()

In [209]:
len(set(ids) - set(ids_full))

121

In [210]:
# Dictionary that links id to parent id.
id_to_parent = df_comments.set_index('id')['parent_comment_id'].to_dict()

In [211]:
# Column with t3 or t1.
df_comments['parent_type'] = df_comments['parent_id'].apply(lambda x: x[:2])

In [212]:
# Divide the datasets in comments of posts and comments of comments.
df_comments_t3 = df_comments[df_comments['parent_type'] == 't3'].reset_index(drop=True)
df_comments_t1 = df_comments[df_comments['parent_type'] == 't1'].reset_index(drop=True)

In [213]:
# If parent post's author is deleted then parent_comment_id = NaN.
df_comments_t3.loc[df_comments_t3['parent_comment_id'].isin(filtered_out_post),'parent_comment_id'] = np.nan

In [214]:
# flag_itp is true if parent_comment_id is not in id_to_parent.keys()
df_comments_t1['flag_itp'] = ~df_comments_t1['parent_comment_id'].isin(id_to_parent.keys())
# flag_foc is true if parent_comment_id is in filtered_out_comm
df_comments_t1['flag_foc'] = df_comments_t1['parent_comment_id'].isin(filtered_out_comm)

while any(df_comments_t1['flag_itp'].to_numpy()) or any(df_comments_t1['flag_foc'].to_numpy()):
    print(f"number of true flag_itp = {sum(df_comments_t1['flag_itp'].to_numpy())}")
    print(f"number of true flag_foc = {sum(df_comments_t1['flag_foc'].to_numpy())}")
    
    # If parent_comment_id is not in id_to_parent then the parent is a deleted post.
    df_comments_t1.loc[df_comments_t1['flag_itp'], 'parent_comment_id'] = np.nan
    df_comments_t1.loc[df_comments_t1['flag_itp'], 'flag_foc'] = False
    # Assign the new parent_comment_id value through id_to_parent.
    df_comments_t1.loc[df_comments_t1['flag_foc'], 'parent_comment_id'] = df_comments_t1.loc[df_comments_t1['flag_foc'], 'parent_comment_id'].apply(lambda x: id_to_parent[x])
    
    # Assign the new flag values.
    df_comments_t1['flag_foc'] = df_comments_t1['parent_comment_id'].isin(filtered_out_comm)
    df_comments_t1['flag_itp'] = ~df_comments_t1['parent_comment_id'].isin(id_to_parent.keys()) & ~df_comments_t1['parent_comment_id'].isna()
    

number of true flag_itp = 68
number of true flag_foc = 724136
number of true flag_itp = 275175
number of true flag_foc = 214434
number of true flag_itp = 76167
number of true flag_foc = 122856
number of true flag_itp = 36785
number of true flag_foc = 78794
number of true flag_itp = 20758
number of true flag_foc = 54115
number of true flag_itp = 12835
number of true flag_foc = 38822
number of true flag_itp = 8308
number of true flag_foc = 28830
number of true flag_itp = 5762
number of true flag_foc = 21860
number of true flag_itp = 4107
number of true flag_foc = 16884
number of true flag_itp = 2982
number of true flag_foc = 13248
number of true flag_itp = 2268
number of true flag_foc = 10496
number of true flag_itp = 1711
number of true flag_foc = 8399
number of true flag_itp = 1299
number of true flag_foc = 6797
number of true flag_itp = 965
number of true flag_foc = 5590
number of true flag_itp = 748
number of true flag_foc = 4645
number of true flag_itp = 589
number of true flag_foc 

In [218]:
# Concatenate the 2 dataframes.
df_comments = pd.concat([df_comments_t3,df_comments_t1])

In [219]:
# Drop all the useless columns.
df_comments = df_comments[df_comments['filter']].drop(columns=['inactivity_filter','flair_filter','deleted_filter','flag_itp','flag_foc'])
df_comments

,author_flair_css_class,created_utc,id,author,parent_id,link_id,filter,parent_comment_id,parent_type
0,NaN,2011-01-11 21:41:34,c1ccls5,Lamzn6,t3_etvwm,t3_etvwm,True,NaN,t3
1,NaN,2011-02-04 08:07:20,c1ffx3l,crystalita,t3_etvwm,t3_etvwm,True,NaN,t3
10,NaN,2011-03-28 01:32:29,c1mmket,Agelity,t3_g1rf3,t3_g1rf3,True,NaN,t3
27,NaN,2011-06-16 15:35:22,c2021y5,crystalita,t3_g1rf3,t3_g1rf3,True,NaN,t3
35,NaN,2011-06-17 00:37:27,c205y6e,crystalita,t3_i1b36,t3_i1b36,True,NaN,t3
...,...,...,...,...,...,...,...,...,...
1633842,infj,2022-12-31 23:39:02,j2fuuj8,Zosi_O,t1_j2fl8v4,t3_10004bx,True,j2fl8v4,t1
1633844,infj,2022-12-31 23:41:11,j2fv4q0,Zosi_O,t1_j2fqyxk,t3_10004bx,True,j2fqyxk,t1
1633845,NaN,2022-12-31 23:47:50,j2fw0zb,Zenishira,t1_j2fv4q0,t3_10004bx,True,j2fv4q0,t1
1633846,estj,2022-12-31 23:48:27,j2fw424,diabolicalfrnchtoast,t1_j2cfdcz,t3_zzlu5n,True,j2cfdcz,t1


In [220]:
# Get all unique ids in posts dataframe.
posts_ids = df_posts.loc[df_posts['filter'],'id'].unique()

In [221]:
# Get all unique ids in comments dataframe.
comments_ids = df_comments.loc[df_comments['filter'],'id'].unique()

In [222]:
# Remove all the rows whit parent_comment_id = NaN.
df_comments = df_comments[~df_comments['parent_comment_id'].isna()]

In [223]:
ids = df_comments['id'].unique()

In [224]:
(set(comments_ids) | set(posts_ids)) - set(id_to_auth.keys())

set()

In [225]:
df_comments[df_comments['id'] == 'v0nxxs']

,author_flair_css_class,created_utc,id,author,parent_id,link_id,filter,parent_comment_id,parent_type


In [226]:
# Create a columns "parent_author" with the author of the parent comment/submission.
print(len(df_comments))
df_comments = df_comments[df_comments['parent_comment_id'].isin(id_to_auth.keys())]
print(len(df_comments))
df_comments['parent_author'] = df_comments['parent_comment_id'].apply(lambda x: id_to_auth[x])

1020760
1020338


In [227]:
# Create a list b in the format needed to build the network.
a = df_comments[['author','parent_author']].values.tolist()

b = []
for elem in a:
    # Don't consider autoloop.
    if elem[0] != elem[1]:
        b.append(f'{elem[0]} {elem[1]}')
        #b.append([elem[0],elem[1]])

In [228]:
# Create a dataframe with edge and its weight.
df_b = pd.DataFrame(b, columns=['edge'])
grouped_df = df_b.groupby(list(df_b.columns)).size().reset_index(name='count')

In [229]:
grouped_df.sort_values('count', ascending=False)

,edge,count
244599,Poropopper CritSrc,199
68556,CritSrc Poropopper,196
73603,DWLlama DaintyDamnation,171
74858,DaintyDamnation DWLlama,152
355259,ZigurotPrime Jamboooo024,98
...,...,...
227086,NuScorpi VenganceNeos1,1
227085,NuScorpi Unspecified66,1
227084,NuScorpi UnhelpfulYodaa,1
227083,NuScorpi UltraEpicSir,1


In [231]:
grouped_df.to_csv("c:/Users/HP/Desktop/UNI/LM_1/SNA/SNA/data/edges_updated.csv")